In [2]:
from dotenv import load_dotenv
load_dotenv()

import os

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "google/gemma-2-9b"  # ローカルまたはHugging Face上のモデルパス
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
)
# トークナイザーのロード
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# モデルのロード（8bit量子化を適用）
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    token=os.getenv("HF_TOKEN"),
)

/home/kaito/anaconda3/envs/llmf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:11<00:00,  1.43s/it]


In [4]:
from datasets import load_dataset

# データセットをロード
dataset = load_dataset("elyza/ELYZA-tasks-100")  # 実際のデータセットパスに変更

# トレーニングデータのみを使用
train_dataset = dataset["test"]

In [4]:
train_dataset[0]

{'input': '仕事の熱意を取り戻すためのアイデアを5つ挙げてください。',
 'output': '1. 自分の仕事に対する興味を再発見するために、新しい技能や知識を学ぶこと。\n2. カレッジやセミナーなどで講演を聴くことで、仕事に対する新しいアイデアや視点を得ること。\n3. 仕事に対してストレスを感じている場合は、ストレスマネジメントのテクニックを学ぶこと。\n4. 仕事以外の楽しいことをすることで、ストレスを発散すること。\n5. 仕事に対して自己評価をすることで、自分がどのように進化しているのかを知ること。',
 'eval_aspect': '- 熱意を取り戻すのではなく、仕事の効率化・スキルアップのような文脈になっていたら1点減点\n- 出したアイデアが5つより多い、少ない場合は1点減点\n- 5つのアイデアのうち、内容が重複しているものがあれば1点減点\n\n'}

In [5]:
def preprocess_function(examples):
    inputs = ["### 入力:\n" + input_text + "\n" for input_text in examples["input"]]
    targets = ["### 出力: " + output_text + "\n\n" for output_text in examples["output"]]
    model_inputs = tokenizer(
        inputs,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    labels = tokenizer(
        targets,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

# データセットのトークナイズと前処理
tokenized_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names,
)

Map: 100%|██████████| 100/100 [00:00<00:00, 2128.38 examples/s]


In [6]:
for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.pre_feedforward_layernorm
model.layers.0.post_feedforward_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.pre_feedforward_layernorm
model.layers.1.post_feedforward_layernorm
model.layers.1.post_atten

In [6]:
from peft import LoraConfig, get_peft_model

# LoRAの設定
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # モデルのアーキテクチャに応じて変更
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [7]:
model.add_adapter(lora_config)

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./lora_gemma_9b",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    report_to="none",
    optim="paged_adamw_8bit",
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# トレーニングの実行
trainer.train()

/tmp/ipykernel_3265843/1106550922.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


/home/kaito/anaconda3/envs/llmf/lib/python3.12/site-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


TrainOutput(global_step=18, training_loss=22.1900634765625, metrics={'train_runtime': 239.728, 'train_samples_per_second': 1.251, 'train_steps_per_second': 0.075, 'total_flos': 7380550268485632.0, 'train_loss': 22.1900634765625, 'epoch': 2.88})

In [10]:
# モデルとトークナイザーの保存
model.save_pretrained("lora_gemma_9b")
tokenizer.save_pretrained("lora_gemma_9b")

/home/kaito/anaconda3/envs/llmf/lib/python3.12/site-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


('lora_gemma_9b/tokenizer_config.json',
 'lora_gemma_9b/special_tokens_map.json',
 'lora_gemma_9b/tokenizer.model',
 'lora_gemma_9b/added_tokens.json')